# Notes

- This is a Python Jupyter notebook but the framework, concepts, and tooling translatest to JS/TS easily
- The live Chatbot demo was built with NextJS/React and Typescript
- This is very code heavy so bear with me

## Dependencies

In [30]:
%pip install langchain faiss-cpu openai beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


# Retrieval Augmented Generation (RAG) - Basic Example

### 1. Creating vector datastores, text splitting, and document creation

In [31]:
# Dependencies
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

# Load the document - this is just a transcript from a recent podcast
# Invest Like the Best with Patrick O'Shaughnessy, Ep Des Traynor - Real Talk about AI and Software - Aug 8, 2023
# https://podcasts.apple.com/us/podcast/des-traynor-real-talk-about-ai-and-software/id1154105909?i=1000623777607
raw_documents = TextLoader('./data/paul-graham-great-work.txt').load()

# Split the document into chunks
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50).split_documents(raw_documents)
print (f"You have {len(documents)} documents")

embeddings = OpenAIEmbeddings()
embedding_list = embeddings.embed_documents([text.page_content for text in documents])
print (f"You have {len(embedding_list)} embeddings")
print (f"Here's a sample of one: {embedding_list[0][:3]}...")

db = FAISS.from_documents(documents, embeddings)

You have 81 documents
You have 81 embeddings
Here's a sample of one: [0.012702980829312127, 0.011727926082949343, 0.01778281026350076]...


### 2. Query our vector datastore directly for relevant documents

In [32]:
query = "What is the key to doing great work?"
relevant_docs = db.similarity_search_with_relevance_scores(query, 3)

print(f"We now have {len(relevant_docs)} relevant docs")
print()

# print the first three docs with a space in between
print("Here is the first doc:")
for doc in relevant_docs[:1]:
    print(doc)
    print()

We now have 3 relevant docs

Here is the first doc:
(Document(page_content="Great work usually entails spending what would seem to most people an unreasonable amount of time on a problem. You can't think of this time as a cost, or it will seem too high. You have to find the work sufficiently engaging as it's happening.\n\nThere may be some jobs where you have to work diligently for years at things you hate before you get to the good part, but this is not how great work happens. Great work happens by focusing consistently on something you're genuinely interested in. When you pause to take stock, you're surprised how far you've come.\n\nThe reason we're surprised is that we underestimate the cumulative effect of work. Writing a page a day doesn't sound like much, but if you do it every day you'll write a book a year. That's the key: consistency. People who do great things don't get a lot done every day. They get something done, rather than nothing.", metadata={'source': './data/paul-grah

### 3. Let's "stuff" these documents into a context so we can query against them

In [35]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}
# openai_api_base='https://llama2-api.main.llama2-api.test1.amazee.io/v1'
qa = RetrievalQA.from_chain_type(llm=OpenAI(openai_api_base='https://llama2-api.main.llama2-api.test1.amazee.io/v1'), chain_type="stuff", retriever=db.as_retriever(), chain_type_kwargs=chain_type_kwargs)


result = qa.run("What are three things anyone can do to achieve great work? Please list them in order of importance as a numbered list.")

In [36]:
from IPython.display import display, Markdown

formatted_text = '\n'.join([f'### {line}' for line in result.strip().split('\n')])

display(Markdown(formatted_text))

### 1) Identify and focus on an area of interest that you have a natural aptitude for and a deep interest in; 2) Maintain curiosity, trying new things, meeting people, reading books, and asking questions to increase your chances of encountering opportunities; 3) Consistently work on this area of interest, even if only making progress at a small rate each day.

---
##

## <[ BACK TO SLIDES ]>

##
---

# RAG Umami Food Blog Chatbot Implementation (Complex Example)
Building on the above steps

1. Setup
2. Data prep/mapping
3. Creating custom documents
4. Leveraging LLM to create custom Tags & add them back to our documents
5. Create a Vectorstore from our documents
6. Introducing "memory" to chatbot context for chat_history
7. Create our "chain" and querying the chatbot


In [6]:
# 1. Setup: Import the necessary libraries, initialize the LLM, and setup the prompt

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv('./.env')

# Get OPENAI_API_KEY and BEARER_TOKEN from .env file
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# initialize LLM (we use ChatOpenAI because we'll later define a `chat` agent)
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name='gpt-3.5-turbo',
)

# Setup the prompt to generate the tags based on the content and specify the format we're looking for
prompt = PromptTemplate.from_template(
"""
You are a content tagging bot for a food and drink blog, and your role is to identify and tag recipes. 
Specify whether the item is a food or drink and then focus on the type (e.g., vegan, gluten-free), unique ingredients (e.g., dried fruit, super seeds), cooking or preparation techniques (e.g., grilling, soaking, mixing), dietary restrictions, cultural origins, meal or occasion types (e.g., breakfast, lunch, dinner, cocktail party), and special flavors or features (e.g., sweet, savory, spicy) that stand out in the given recipe.

Recipe: {recipe}

Feel free to add any tags that may provide insightful information about the dish or drink. There's no maximum number of tags, so be thorough and descriptive in your tagging, as it helps readers find recipes that match their preferences.
YOU MUST return the tags in the following format:

Category: Drink, Type: Vegan, Unique Ingredients: Mint leaves, Preparation Techniques: Mixing, Dietary Restrictions: Gluten-free, Cultural Origins: Cuban, Occasion: Cocktail party, Special Features: Refreshing]

"""
)

# put the prompt together with the LLM we want to use
chain = LLMChain(llm=llm, prompt=prompt)

In [7]:
## 2 & 3. Data Prep/Mapping, Creating Custom Documents - Load the JSON API export, clean the HTML, and create a Document object for each page

from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from bs4 import BeautifulSoup
import asyncio
import json

# Load our data expore - could be a URL or a local file
with open('./ingest/composetheweb.json', 'r') as file:
    jsonapi_export = json.load(file)


# Clean the HTML from the JSON API export
def clean_html(html_content):
    return BeautifulSoup(html_content, 'html.parser').get_text()

# Create and format the document object
def create_document(doc):
    title = doc['attributes']['title']
    source = doc['attributes']['path']['alias']
    metadata = {'title': title, 'source': source}
    
    if doc['type'] == 'node--recipe':
        difficulty = str(doc['attributes']['field_difficulty'])
        ingredients = str(doc['attributes']['field_ingredients'])
        recipe = clean_html(str(doc['attributes']['field_recipe_instruction']['value']))
        summary = clean_html(str(doc['attributes']['field_summary']['value']))
        
        page_content = f"Title: {title},\nDifficulty: {difficulty},\n\nIngredients: {ingredients},\n\nRecipe: {recipe}, \n\nSummary: {summary}"
    else:
        body = clean_html(str(doc['attributes']['body']['value']))
        page_content = f"Title: {title} - {body}"
    
    return Document(metadata=metadata, page_content=page_content)

# loop over the JSON API export and create a Document object for each page
docs = [create_document(doc) for doc in jsonapi_export['data']]

In [10]:
## DO NOT RUN DURING DEMO - ~ 1-2 min to run

# 4. Leveraging LLM to create custom Tags & add them back to our documents - Why? See Side Note below...

# Simple wrapper function around the chain invocation
async def generateTagsFromContent(content):
    return chain.invoke({"recipe": content})

# Run the documents back through a LLM to generate tags
generateTagPromises = [generateTagsFromContent(doc.page_content) for doc in docs]
tags = await asyncio.gather(*generateTagPromises)

# loop over the tags and add them to the metadata
updatedDocs = []
for index, doc in enumerate(docs):
    updatedDocs.append(
        Document(
            metadata={**doc.metadata, 'tags': tags[index]['text']}, 
            page_content=doc.page_content
        )
    )


In [11]:
# 5. Create a Vectorstore from our documents

print("Creating vector store...")
vectorstore: FAISS = FAISS.from_documents(updatedDocs, OpenAIEmbeddings())
vectorstore.save_local("data")

Creating vector store...


In [34]:
# Recap - Let's take a peek at the data in our vectorstore
print(f"Number of documents: {len(updatedDocs)}")
print()
print("Example Document Content: (First 300 chars)")
print(updatedDocs[8].page_content[:300])
print('------------------')
print("Metadata object containing our source and generated tags")
print(updatedDocs[8].metadata)

Number of documents: 18

Example Document Content: (First 300 chars)
Title: Borscht with pork ribs,
Difficulty: medium,

Ingredients: ['400-500g pork ribs', '2 beets', '2 tomatoes', '¼ celery root', '¼ cabbage', '3-4 medium potatoes', '1 carrot', '1 onion', '1-2 smoked pears', '2 bay leaves', '3 allspice berries', '1 bulb garlic', '1 bell pepper', '200ml tomato juice
------------------
Metadata object containing our source and generated tags
{'title': 'Borscht with pork ribs', 'source': '/recipes/borscht-with-pork-ribs', 'tags': 'Category: Food, Type: Non-vegetarian, Unique Ingredients: Pork ribs, Smoked pears, Preparation Techniques: Baking, Simmering, Sautéing, Stewing, Dietary Restrictions: None, Cultural Origins: Ukrainian, Occasion: Lunch, Dinner, Special Features: Hearty, Flavorful, Savory]'}


In [22]:
# Side Note: Why have an LLM create tags?
# Generated tags allow us to create a much more useful and broad search of our content

# Example - Ukrainian Dishes are not mentioned directly in the recipe at all. 
# Searching for this via a traditional Drupal or Solr search would not be possible without manually adding tags like this
question = "What Ukranian dishes do we have?"

found_docs = vectorstore.similarity_search_with_relevance_scores(question )
print(f"{found_docs[0][0].page_content[:100]}...")
print(f"Metadata: {found_docs[0][0].metadata}")
print(f"Relavance Score: {found_docs[0][1]}")

Title: Borscht with pork ribs,
Difficulty: medium,

Ingredients: ['400-500g pork ribs', '2 beets', '...
Metadata: {'title': 'Borscht with pork ribs', 'source': '/recipes/borscht-with-pork-ribs', 'tags': 'Category: Food, Type: Non-vegetarian, Unique Ingredients: Pork ribs, Smoked pears, Preparation Techniques: Baking, Simmering, Sautéing, Stewing, Dietary Restrictions: None, Cultural Origins: Ukrainian, Occasion: Lunch, Dinner, Special Features: Hearty, Flavorful, Savory]'}
Relavance Score: 0.6947289853895937


## Chains & Memory

What is a chain?

"Chain" - Langchain Framework Construct: Just the ability to combine multiple LLMs in series and/or with logic applied
This allows us to implement much deeper systems and workflows.

Example: Combine Vector Search + Question, Follow up question w/ original Vector Search + Conversation history, Etc...

### (Advanced Chain)  ConversationalRetrievalChain

In [24]:
# 6. Introducing "memory" to chatbot context for chat_history

from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)


llm = OpenAI(temperature=0)
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)

# note we get the sources w/ this chain
doc_chain = load_qa_with_sources_chain(llm, chain_type="stuff")

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [26]:
# 7. Create our "chain" and querying the chatbot

chain = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    memory=memory,
)

chat_history = []
query = "Do we have any Ukranian dishes? Please respond with just the title and a single sentence description."
result = chain({"question": query})

print(f"{result['answer'][:500]}...")

 Borscht is a traditional Ukrainian dish made with pork ribs, beets, tomatoes, celery root, cabbage, potatoes, carrot, onion, smoked pears, bay leaves, allspice berries, garlic, bell pepper, tomato juice, butter, tomato paste, water, sour cream, and beans (optional).
SOURCES: /recipes/borscht-with-pork-ribs...


In [27]:
query = "Can you provide me with the full recipe?"
result = chain(query)

print(result['answer'])

 The full recipe for Borscht with pork ribs is:
400-500g pork ribs, 2 beets, 2 tomatoes, ¼ celery root, ¼ cabbage, 3-4 medium potatoes, 1 carrot, 1 onion, 1-2 smoked pears, 2 bay leaves, 3 allspice berries, 1 bulb garlic, 1 bell pepper, 200ml tomato juice, 30g butter, 2 tbsp tomato paste, 3l water, Sour cream, 1 can beans (optional), Salt to taste.

Preheat the oven to 400°F/200°C. Put pork ribs on the baking dish and bake them for 30 minutes. Wash but don’t peel the celery root. Dice the carrot. Put the baked pork ribs in a pot and add 3 liters of water. Chop the celery root and carrot and add them to the pot, along with half the unpeeled onion. Bring the pot to the boil and simmer for 30 minutes. The heart of the borscht is sautéed vegetables. Dice the bell pepper, tomatoes, and remainder of the onion. Place the butter in a saucepan and add the vegetables. Cook them over a moderate heat until soft. Add the tomato paste, reduce the


In [28]:
# pretty print the chat history object
from pprint import pprint

pprint(result['chat_history'], indent=2, width=60)

[ HumanMessage(content='Do we have any Ukranian dishes? Please respond with just the title and a single sentence description.', additional_kwargs={}, example=False),
  AIMessage(content=' Title: Borscht with pork ribs, Description: A traditional Ukrainian dish made with pork ribs, beets, tomatoes, celery root, cabbage, potatoes, carrot, onion, smoked pears, bay leaves, allspice berries, garlic, bell pepper, tomato juice, butter, tomato paste, and water.\nSOURCES: /recipes/borscht-with-pork-ribs', additional_kwargs={}, example=False),
  HumanMessage(content='Do we have any Ukranian dishes? Please respond with just the title and a single sentence description.', additional_kwargs={}, example=False),
  AIMessage(content=' Borscht is a traditional Ukrainian dish made with pork ribs, beets, tomatoes, celery root, cabbage, potatoes, carrot, onion, smoked pears, bay leaves, allspice berries, garlic, bell pepper, tomato juice, butter, tomato paste, water, sour cream, and beans (optional).\nSOUR

## LLama2

In [ ]:
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
import requests
from typing import Any, List, Mapping, Optional
from langchain.llms.base import LLM

class RemoteLLM(LLM):
    url: str

    @property
    def _llm_type(self) -> str:
        return "remote"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")


        headers = {"Content-Type": "application/json"}
        body = {
            "model": "gpt-3.5-turbo",
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.7,
            "max_tokens": 2000
        }
        
        response = requests.post(self.url, json=body, headers=headers)
        
        if response.status_code != 200:
            raise Exception("Request failed with status code:", response.status_code)
        
        response_content = response.json().get('choices', [{}])[0].get('message', {}).get('content', '')
        return response_content

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"url": self.url}


llm = RemoteLLM(url="https://llama2-api.main.llama2-api.test1.amazee.io/v1/chat/completions")
response = llm("What's the biggest animal in the world?")
print(response)
